In [13]:
import os
import requests
import numpy as np
import soundfile as sf
import librosa
from scipy.io.wavfile import write
import IPython.display as ipd
from IPython.display import display
import subprocess
import sys
import json
import warnings
from scipy.signal import butter, lfilter, sosfilt
import noisereduce as nr

warnings.filterwarnings("ignore")


try:
    import torch
    from transformers import VitsModel, AutoTokenizer
    print("PyTorch and Transformers loaded.")

except ImportError:
    print("PyTorch or Transformers library not found.")
    torch = None
    VitsModel = None
    AutoTokenizer = None


# Install dependencies
def install_dependencies():
    packages = [
        "soundfile",
        "librosa",
        "scipy",
        "numpy",
        "requests",
        "ipython",
        "torch",
        "transformers",
        "accelerate" # Direkomendasikan oleh Hugging Face
        "noisereduce"
    ]

    for package in packages:
        try:
            print(f"Installing {package}...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        except Exception as e:
            print(f"Failed to install {package}: {e}")

    print("Dependencies installation completed!")




class MMSTTSEngine:
    """
    MMS TTS Engine untuk HiTeman Audio Therapy
    Menggunakan model VITS dari Hugging Face (facebook/mms-tts-ind)
    dengan tambahan post-processing (v2) untuk kejernihan suara.
    """

    def __init__(self):
        print(f"Initializing MMS TTS Engine (Indonesian) Post-Processing (v2)..")
        self.model_id = "facebook/mms-tts-ind"
        self.model = None
        self.tokenizer = None
        self.device = "cpu"
        self.sample_rate = 16000

        if torch and VitsModel and AutoTokenizer:
            try:
                self.device = "cuda" if torch.cuda.is_available() else "cpu"
                print(f"Loading MMS TTS model '{self.model_id}' onto {self.device}...")
                self.tokenizer = AutoTokenizer.from_pretrained(self.model_id)
                self.model = VitsModel.from_pretrained(self.model_id).to(self.device)
                self.sample_rate = self.model.config.sampling_rate
                print(f"MMS TTS model loaded successfully. Sample rate: {self.sample_rate} Hz")
            except Exception as e:
                print(f"Failed to load MMS TTS model: {e}")
                print("Fallback to basic TTS will be used.")
                self.model = None
        else:
            print("PyTorch or Transformers not available. MMS TTS cannot be loaded.")
            print("Fallback to basic TTS will be used.")

    def _butter_highpass(self, cutoff, fs, order=5):
        nyq = 0.5 * fs
        normal_cutoff = cutoff / nyq
        b, a = butter(order, normal_cutoff, btype='high', analog=False)
        return b, a

    def _butter_highpass_filter(self, data, cutoff, fs, order=5):
        b, a = self._butter_highpass(cutoff, fs, order=order)
        y = lfilter(b, a, data)
        return y

    def _peaking_filter_coeffs(self, freq_hz, Q, gain_db, fs):
        """
        Menghasilkan koefisien SOS untuk filter peaking EQ.

        """
        A = 10**(gain_db / 40.0)
        w0 = 2 * np.pi * freq_hz / fs
        alpha = np.sin(w0) / (2.0 * Q)

        b0 = 1 + alpha * A
        b1 = -2 * np.cos(w0)
        b2 = 1 - alpha * A
        a0_denom = 1 + alpha / A # Ini akan menjadi pembagi untuk normalisasi a0 ke 1
        a1 = -2 * np.cos(w0)
        a2 = 1 - alpha / A

        # Koefisien dinormalisasi agar a0 = 1 untuk sosfilt
        # Format SOS: [b0, b1, b2, a0 (selalu 1 setelah normalisasi), a1, a2]
        sos_coeffs = np.array([b0/a0_denom, b1/a0_denom, b2/a0_denom,
                               1.0,         a1/a0_denom, a2/a0_denom])
        return sos_coeffs.reshape(1, 6) # sosfilt mengharapkan array 2D (n_sections, 6)

    def _apply_peaking_filter(self, data, freq_hz, Q, gain_db, fs):
        sos = self._peaking_filter_coeffs(freq_hz, Q, gain_db, fs)
        y = sosfilt(sos, data)
        return y

    def synthesize_speech(self, text, voice_style="neutral", speed=1.0):
        if not self.model or not self.tokenizer:
            print("MMS TTS model not loaded, using basic TTS.")
            return self._generate_basic_tts(text, speed)

        try:
            print(f"Synthesizing with MMS TTS: '{text[:50]}...'")
            inputs = self.tokenizer(text, return_tensors="pt").to(self.device)
            with torch.no_grad():
                output = self.model(**inputs).waveform

            if output is None:
                print(" MMS TTS model generated None waveform.")
                return self._generate_basic_tts(text, speed)

            audio_data = output.squeeze().cpu().numpy()

            if not isinstance(audio_data, np.ndarray) or audio_data.ndim != 1:
                print(f"Unexpected audio data format from MMS TTS.")
                return self._generate_basic_tts(text, speed)

            current_sample_rate = self.sample_rate
            print(f"MMS Audio generated: {len(audio_data)} samples at {current_sample_rate} Hz")

            # --- POST-PROCESSING STEPS (v2) ---
            print("Applying post-processing (v2)")

            # 1. Noise Reduction (parameter sedikit diubah)
            if nr:
                try:
                    audio_data_float = audio_data.astype(np.float32)
                    if len(audio_data_float) > current_sample_rate * 0.1:
                        reduced_noise_audio = nr.reduce_noise(
                            y=audio_data_float,
                            sr=current_sample_rate,
                            prop_decrease=0.7,       # Coba antara 0.65 - 0.8. Lebih rendah = kurang agresif.
                            n_fft=512,               # Ukuran window FFT
                            hop_length=128,          # Hop length (biasanya n_fft/4)
                            n_std_thresh_stationary=1.3, # Threshold untuk noise stasioner (1.0-1.5)
                            # stationary=False, # Coba set ke False jika noise tidak stasioner
                            freq_mask_smooth_hz=300, # Kurangi smoothing frekuensi sedikit
                            time_mask_smooth_ms=40   # Kurangi smoothing waktu sedikit
                        )
                        audio_data = reduced_noise_audio
                        print(" Noise reduction (v2) applied.")
                    else:
                        print(" Audio too short for effective noise reduction, skipping.")
                except Exception as e:
                    print(f"  Error during noise reduction (v2): {e}. Skipping.")
            else:
                print("  Noisereduce library not available, skipping noise reduction.")

            # 2. High-pass Filter (menghilangkan frekuensi rendah yang tidak diinginkan)
            try:
                cutoff_freq = 90.0 # Hz (antara 80-100 Hz)
                audio_data = self._butter_highpass_filter(audio_data, cutoff_freq, current_sample_rate, order=4)
                print(f"   High-pass filter applied (cutoff: {cutoff_freq} Hz).")
            except Exception as e:
                print(f"   Error during high-pass filtering: {e}. Skipping.")

            # 3. Vocal Clarity Boost (Equalization - Peaking Filter)
            try:
                # Tingkatkan frekuensi di sekitar area presence/clarity vokal
                # Freq: Frekuensi tengah (Hz)
                # Q: Faktor Q (mengontrol lebar pita; nilai lebih tinggi = pita lebih sempit)
                # Gain_db: Penguatan dalam dB (positif untuk boost, negatif untuk cut)

                # Boost 1: Upper Midrange/Presence untuk kejelasan artikulasi
                clarity_freq1 = 3000.0  # Hz (coba antara 2500 - 4000 Hz, jgn terlalu tinggi)
                clarity_Q1 = 1.8        # Q (coba antara 1.5 - 2.5, jgn terlalu tinggi)
                clarity_gain_db1 = 2.0  # dB (coba antara 1.5 - 3.0 dB, jangan terlalu tinggi)
                audio_data = self._apply_peaking_filter(audio_data, clarity_freq1, clarity_Q1, clarity_gain_db1, current_sample_rate)
                print(f"  Vocal Clarity EQ 1 applied (Freq: {clarity_freq1}Hz, Q: {clarity_Q1}, Gain: {clarity_gain_db1}dB).")

                # (Opsional) Boost 2: Sedikit di lower midrange untuk "body" jika suara terlalu tipis
                # body_freq = 500.0       # Hz (coba antara 400 - 600 Hz)
                # body_Q = 1.0
                # body_gain_db = 1.0    # dB (boost kecil saja)
                # audio_data = self._apply_peaking_filter(audio_data, body_freq, body_Q, body_gain_db, current_sample_rate)
                # print(f"Vocal Body EQ applied (Freq: {body_freq}Hz, Q: {body_Q}, Gain: {body_gain_db}dB).")

                # (Opsional) Cut: Jika ada frekuensi "boxy" atau "honky" (biasanya di mid-low mid)
                # boxy_freq = 400.0       # Hz (coba antara 300 - 600 Hz)
                # boxy_Q = 1.5
                # boxy_gain_db = -1.5   # dB (cut kecil)
                # audio_data = self._apply_peaking_filter(audio_data, boxy_freq, boxy_Q, boxy_gain_db, current_sample_rate)
                # print(f"Boxy Frequency Cut EQ applied (Freq: {boxy_freq}Hz, Q: {boxy_Q}, Gain: {boxy_gain_db}dB).")

            except Exception as e:
                print(f"Error during vocal clarity EQ: {e}. Skipping.")

            # --- END OF POST-PROCESSING ---

            if speed != 1.0:
                if speed <= 0: print(f"Invalid speed value {speed}. Using 1.0.")
                else:
                    try:
                        audio_data = librosa.effects.time_stretch(audio_data, rate=speed)
                        print(f"Speed adjusted to {speed}x")
                    except Exception as e: print(f"Failed to adjust speed: {e}. Using original speed.")

            max_val = np.max(np.abs(audio_data))
            if max_val > 0: audio_data = audio_data / max_val * 0.88 # Normalisasi sedikit lebih tinggi

            print("Post-processing (v2) complete.")
            return audio_data.astype(np.float32), current_sample_rate

        except Exception as e:
            print(f"MMS TTS synthesis with post-processing (v2) failed: {e}")
            return self._generate_basic_tts(text, speed)

    def _generate_basic_tts(self, text, speed):
        # ... (Implementasi _generate_basic_tts tetap sama seperti sebelumnya) ...
        print("Using enhanced basic TTS synthesis...")
        estimated_syllables = len(text.split()) * 2
        duration = max(estimated_syllables / (3.5 * speed), 1.5)
        basic_sample_rate = 22050
        t = np.linspace(0, duration, int(basic_sample_rate * duration), endpoint=False)
        f0_base = 180
        pitch_contour = np.sin(2 * np.pi * 0.3 * t) * 30
        sentence_intonation = np.sin(2 * np.pi * 0.8 * t) * 15
        f0 = f0_base + pitch_contour + sentence_intonation
        harmonics_amps = [0.8, 0.3, 0.15, 0.08, 0.04]
        audio = sum(amp * np.sin(2 * np.pi * f0 * (i+1) * t) for i, amp in enumerate(harmonics_amps))
        formant_freqs = [700, 1100, 2400]
        formant_amps = [0.25, 0.15, 0.08]
        for freq, amp in zip(formant_freqs, formant_amps):
            formant = amp * np.sin(2 * np.pi * freq * t)
            formant *= (1 + 0.2 * np.sin(2 * np.pi * f0 * t))
            audio += formant
        word_count = len(text.split())
        if word_count > 5:
            num_pauses = max(1, word_count // 5)
            pause_positions = np.linspace(0.2, 0.8, num_pauses) if num_pauses > 0 else []
            for pos_norm in pause_positions:
                pause_center_idx = int(pos_norm * len(t))
                pause_width_samples = int(0.12 * basic_sample_rate)
                start_idx = max(0, pause_center_idx - pause_width_samples // 2)
                end_idx = min(len(audio), pause_center_idx + pause_width_samples // 2)
                fade_samples = pause_width_samples // 4
                if start_idx + fade_samples < end_idx - fade_samples and fade_samples > 0:
                    audio[start_idx : start_idx + fade_samples] *= np.linspace(1, 0.2, fade_samples)
                    audio[start_idx + fade_samples : end_idx - fade_samples] *= 0.2
                    audio[end_idx - fade_samples : end_idx] *= np.linspace(0.2, 1, fade_samples)
        breath_pattern = 0.92 + 0.08 * np.sin(2 * np.pi * 0.4 * t)
        emphasis_pattern = 1 + 0.15 * np.sin(2 * np.pi * 1.8 * t)
        audio *= breath_pattern * emphasis_pattern
        attack_time, release_time = 0.05, 0.1
        attack_samples = int(attack_time * basic_sample_rate)
        release_samples = int(release_time * basic_sample_rate)
        envelope = np.ones_like(audio)
        if len(envelope) > attack_samples : envelope[:attack_samples] = np.linspace(0, 1, attack_samples)**2
        if len(envelope) > release_samples : envelope[-release_samples:] = np.linspace(1, 0, release_samples)**2
        audio *= envelope
        noise = np.random.normal(0, 0.002, len(audio))
        audio += noise
        max_val = np.max(np.abs(audio))
        if max_val > 0: audio = (audio / max_val) * 0.75
        audio = np.tanh(audio * 1.1) / 1.1
        print(f" Enhanced basic TTS generated: {duration:.1f}s at {basic_sample_rate} Hz")
        return audio.astype(np.float32), basic_sample_rate



class AudioTherapyGenerator:
    """Generator untuk audio therapy dengan MMS TTS"""

    def __init__(self, tts_engine):
        self.tts = tts_engine
        self.therapy_scripts = self.load_therapy_scripts()

    def load_therapy_scripts(self):
        """Load therapy scripts dalam bahasa Indonesia yang natural"""
        # Script tetap sama
        return {
            "sedih": [
                "Halo, saya memahami bahwa Anda sedang merasa sedih saat ini. Perasaan sedih adalah bagian alami dari kehidupan manusia, dan Anda tidak sendirian dalam menghadapinya.",
                "Mari kita mulai dengan latihan pernapasan yang sederhana. Tarik napas dalam-dalam melalui hidung selama empat hitungan, tahan sejenak, lalu buang napas perlahan melalui mulut selama enam hitungan.",
                "Ingatlah bahwa kesedihan ini tidak akan berlangsung selamanya. Seperti cuaca yang berubah, perasaan Anda juga akan berubah. Berikan waktu untuk diri Anda merasakan emosi ini tanpa menghakimi.",
                "Cobalah untuk melakukan satu aktivitas kecil yang biasanya memberikan Anda ketenangan. Mungkin mendengarkan musik favorit, melihat foto-foto indah, atau sekadar minum teh hangat dengan perlahan.",
                "Anda memiliki kekuatan untuk melewati masa sulit ini. Di dalam diri Anda ada ketahanan yang mungkin belum Anda sadari sepenuhnya. Percayalah pada proses penyembuhan Anda."
            ],
            "cemas": [
                "Saya memahami bahwa kecemasan yang Anda rasakan saat ini terasa sangat nyata dan mengganggu. Kecemasan adalah cara tubuh kita mempersiapkan diri menghadapi bahaya, meski terkadang sinyal ini muncul saat tidak diperlukan.",
                "Mari kita praktikkan teknik grounding lima-empat-tiga-dua-satu. Sebutkan dalam hati lima hal yang dapat Anda lihat di sekitar Anda sekarang. Kemudian empat hal yang dapat Anda sentuh, tiga hal yang dapat Anda dengar, dua hal yang dapat Anda cium, dan satu hal yang dapat Anda rasakan.",
                "Pernapasan adalah jangkar kita saat kecemasan menyerang. Letakkan satu tangan di dada dan satu di perut. Bernapaslah sehingga tangan di perut bergerak lebih banyak daripada tangan di dada. Ini adalah pernapasan dari diafragma yang akan menenangkan sistem saraf Anda.",
                "Ingatlah bahwa pikiran cemas bukanlah fakta atau kenyataan. Mereka hanyalah pikiran yang datang dan pergi seperti awan di langit. Anda tidak harus percaya atau bertindak berdasarkan setiap pikiran yang muncul.",
                "Kecemasan sering kali membuat kita fokus pada masa depan yang belum terjadi. Cobalah untuk kembali ke momen saat ini. Rasakan kaki Anda menyentuh lantai, dengarkan suara di sekitar Anda, sadari bahwa saat ini Anda aman."
            ],
            "stress": [
                "Saya dapat merasakan bahwa Anda sedang mengalami tekanan yang cukup berat. Stres adalah sinyal dari tubuh bahwa kita perlu memberikan perhatian lebih pada diri kita sendiri dan cara kita mengelola tuntutan hidup.",
                "Mari kita lakukan relaksasi otot progresif. Mulai dari kaki Anda, kencangkan otot-otot selama lima detik, lalu lepaskan dengan perlahan. Rasakan perbedaan antara ketegangan dan relaksasi. Lanjutkan ke betis, paha, perut, tangan, lengan, bahu, dan wajah.",
                "Penting untuk mengingat bahwa tidak semua hal harus diselesaikan hari ini. Cobalah untuk membuat daftar prioritas dan fokus pada satu tugas dalam satu waktu. Memberikan perhatian penuh pada satu hal akan lebih efektif daripada mencoba melakukan banyak hal sekaligus.",
                "Berikan diri Anda izin untuk beristirahat tanpa merasa bersalah. Istirahat bukanlah tanda kelemahan atau kemalasan, melainkan investasi untuk kesehatan mental dan produktivitas jangka panjang Anda.",
                "Ingatlah untuk menjaga keseimbangan antara memberikan yang terbaik dan menjaga kesejahteraan diri. Anda adalah manusia, bukan mesin. Nilai diri Anda tidak hanya ditentukan oleh pencapaian atau produktivitas, tetapi juga oleh kebaikan Anda terhadap diri sendiri."
            ]
        }

    def generate_therapy_audio(self, condition, session_length="medium"):
        """Generate audio therapy berdasarkan kondisi mental"""
        if condition not in self.therapy_scripts:
            print(f"Kondisi '{condition}' tidak ditemukan, menggunakan 'sedih' sebagai default.")
            condition = "sedih"

        scripts = self.therapy_scripts[condition]

        if session_length == "short": selected_scripts = scripts[:2]
        elif session_length == "medium": selected_scripts = scripts[:4]
        else: selected_scripts = scripts

        audio_files = []
        voice_params = self.get_voice_params(condition) # Ini masih mengatur 'speed'

        print(f"🎵 Generating {len(selected_scripts)} audio files for {condition} therapy using MMS TTS...")

        for i, script in enumerate(selected_scripts):
            print(f"   Creating audio {i+1}/{len(selected_scripts)}...")

            try:
                # voice_style tidak lagi relevan untuk MMS TTS, tapi speed masih bisa digunakan
                audio_data, sample_rate = self.tts.synthesize_speech(
                    text=script,
                    voice_style="neutral", # Tidak digunakan oleh MMSTTSEngine tapi ada di signature
                    speed=voice_params["speed"]
                )

                if audio_data is not None and sample_rate is not None:
                    filename = f"therapy_mms_{condition}_{i+1}.wav" # Ubah nama file untuk bedakan
                    sf.write(filename, audio_data, sample_rate)
                    audio_files.append(filename)
                    print(f"   ✅ Saved: {filename} (Sample Rate: {sample_rate} Hz)")
                else:
                    print(f"   ❌ Failed to generate audio {i+1} (received None).")

            except Exception as e:
                print(f"   ❌ Failed to generate audio {i+1}: {e}")
                continue

        print(f"🎵 Therapy audio generation complete! Created {len(audio_files)} files.")
        return audio_files

    def get_voice_params(self, condition):
        """Dapatkan parameter suara berdasarkan kondisi emosional (hanya speed yang relevan untuk MMS)"""
        params = {
            "sedih": {"style": "neutral", "speed": 0.95}, # Speed bisa disesuaikan
            "cemas": {"style": "neutral", "speed": 0.9},
            "stress": {"style": "neutral", "speed": 0.98}
        }
        return params.get(condition, params["sedih"])


class HiTemanAudioTherapy:
    """Sistem Audio Therapy lengkap dengan MMS TTS"""

    def __init__(self): # Tidak perlu API key lagi
        print("🚀 Initializing HiTeman Audio Therapy System menggunakan MMS TTS...")


        self.tts_engine = MMSTTSEngine() # Menggunakan engine MMS TTS
        self.therapy_generator = AudioTherapyGenerator(self.tts_engine)
        print("✅ HiTeman Audio Therapy System (MMS TTS) ready!")

    def show_greeting_text(self):
        """Tampilkan teks greeting"""
        greeting_text = """
        🌟 SELAMAT DATANG DI HITEMAN AUDIO THERAPY 🌟

        Halo! Saya adalah asisten terapi audio digital yang akan membantu Anda
        menemukan ketenangan dan dukungan untuk kesehatan mental Anda.

        Sistem ini menggunakan teknologi AI voice (MMS TTS) untuk memberikan:
        • Audio terapi dengan suara natural dalam bahasa Indonesia
        • Panduan pernapasan dan relaksasi
        • Dukungan positif

        Mari kita mulai perjalanan menuju kesejahteraan mental yang lebih baik!
        """
        print(greeting_text)
        return greeting_text

    def show_screening_questions(self):
        """Tampilkan screening questions (tetap sama)"""
        questions = [
            "1. Bagaimana perasaan Anda secara umum hari ini? (sedih/cemas/stress/baik)",
            "2. Apakah Anda merasa kesulitan untuk berkonsentrasi?",
            "3. Seberapa sering Anda merasa khawatir dalam seminggu terakhir?",
            "4. Apakah Anda merasa terbebani dengan tanggung jawab sehari-hari?",
            "5. Bagaimana kualitas tidur Anda dalam beberapa hari terakhir?"
        ]

        print("\n📋 SCREENING QUESTIONS:")
        print("Silakan jawab pertanyaan berikut untuk menentukan jenis terapi yang tepat:\n")

        for question in questions:
            print(question)

        print("\nBerdasarkan jawaban Anda, pilih kondisi yang paling sesuai:")
        print("• 'sedih' - jika Anda merasa down, kehilangan motivasi, atau murung")
        print("• 'cemas' - jika Anda merasa khawatir berlebihan, gelisah, atau panik")
        print("• 'stress' - jika Anda merasa kewalahan, tertekan, atau terbebani\n")

        return questions

    def generate_therapy_session(self, mental_condition, session_length="medium"):
        """Generate complete therapy session audio"""
        print(f" Starting therapy session for: {mental_condition.upper()}")
        print(f"📏 Session length: {session_length}")

        intro_text = f"""
        🎵 SESI TERAPI AUDIO - {mental_condition.upper()} 🎵

        Berdasarkan kondisi yang Anda alami, saya akan memberikan panduan terapi audio
        yang dirancang khusus untuk membantu mengatasi {mental_condition}.

        Silakan:
        • Duduk atau berbaring dengan nyaman
        • Gunakan headphone untuk pengalaman terbaik
        • Ikuti panduan yang diberikan dengan santai
        • Jangan memaksakan diri, lakukan sesuai kemampuan

        Mari kita mulai...
        """
        print(intro_text)

        therapy_files = self.therapy_generator.generate_therapy_audio(
            condition=mental_condition,
            session_length=session_length
        )

        closing_text = """
         PENUTUP SESI TERAPI

        Terima kasih telah mengikuti sesi terapi audio ini. Ingatlah:

        • Perubahan membutuhkan waktu dan latihan yang konsisten
        • Setiap langkah kecil adalah kemajuan yang berarti
        • Anda tidak sendirian dalam perjalanan ini
        • Jangan ragu untuk kembali kapan saja Anda membutuhkan dukungan

        Semoga Anda merasa lebih tenang dan memiliki tools untuk menghadapi hari-hari ke depan.
        Jaga diri Anda dengan baik! 💚
        """
        print(closing_text)

        session_info = {
            "condition": mental_condition,
            "session_length": session_length,
            "intro_text": intro_text,
            "closing_text": closing_text,
            "therapy_files": therapy_files,
            "total_files": len(therapy_files)
        }

        print(f"✅ Therapy session complete! Generated {len(therapy_files)} audio files.")
        return session_info

    def play_audio(self, filename):
        """Play audio file (tetap sama)"""
        if os.path.exists(filename):
            print(f"🔊 Playing: {filename}")
            return ipd.Audio(filename)
        else:
            print(f"❌ Audio file not found: {filename}")
            return None

    def demo_therapy_system(self):
        """Demo sistem terapi audio lengkap"""
        print("="*60)
        print("🌟 HITEMAN AUDIO THERAPY SYSTEM DEMO (MMS TTS) 🌟")
        print("="*60)

        self.show_greeting_text()
        print("\n" + "-"*60 + "\n")
        self.show_screening_questions()
        print("-"*60 + "\n")

        conditions = ["sedih", "cemas", "stress"]

        for i, condition in enumerate(conditions, 1):
            print(f"🎵 DEMO {i}: THERAPY SESSION - {condition.upper()}")
            print("-"*40)

            session_info = self.generate_therapy_session(condition, "short")

            if session_info["therapy_files"]:
                first_audio = session_info["therapy_files"][0]
                print(f"\n🎧 Demo audio untuk {condition} (file pertama):")
                audio_player = self.play_audio(first_audio)
                if audio_player:
                    try:
                        display(audio_player)
                    except NameError:
                        print("   (Untuk memutar audio, jalankan di lingkungan yang mendukung IPython.display seperti Jupyter Notebook)")
                        print(f"   Representasi objek audio: {audio_player}")


                print(f"📁 Generated files for {condition}:")
                for file_path in session_info["therapy_files"]:
                    print(f"   • {file_path}")
            else:
                 print(f"⚠️ No audio files generated for {condition}.")
            print("\n" + "="*60 + "\n")

        print(" COMPLETE!")
        print("="*60)


# Usage Examples
if __name__ == "__main__":
    print("============================================================")
    print("Selamat datang di HiTeman - Audio Therapy dengan MMS TTS")
    print("============================================================")

    print("Script ini akan mendemonstrasikan sistem terapi audio menggunakan MMS TTS.")
    print("Pastikan Anda memiliki koneksi internet saat pertama kali menjalankan")
    print("untuk mengunduh model MMS TTS dari Hugging Face (sekitar 250MB).")
    print("-" * 60)


    # Inisialisasi sistem terapi
    therapy_system = HiTemanAudioTherapy()

    # Jalankan demo sistem
    try:
        therapy_system.demo_therapy_system()
    except NameError as e:
        if 'display' in str(e):
            print("\n⚠️  PERINGATAN: Fungsi 'display' tidak tersedia.")
        else:
            print(f"\n❌ Terjadi kesalahan NameError saat menjalankan demo: {e}")
    except Exception as e:
        print(f"\n❌ Terjadi kesalahan umum saat menjalankan demo: {e}")

    print("\n============================================================")
    print("Demo HiTeman Audio Therapy (MMS TTS) Selesai.")
    print("Terima kasih telah menggunakan HiTeman!")
    print("============================================================")

PyTorch and Transformers loaded.
Selamat datang di HiTeman - Audio Therapy dengan MMS TTS
Script ini akan mendemonstrasikan sistem terapi audio menggunakan MMS TTS.
Pastikan Anda memiliki koneksi internet saat pertama kali menjalankan
untuk mengunduh model MMS TTS dari Hugging Face (sekitar 250MB).
------------------------------------------------------------
🚀 Initializing HiTeman Audio Therapy System menggunakan MMS TTS...
Initializing MMS TTS Engine (Indonesian) Post-Processing (v2)..
Loading MMS TTS model 'facebook/mms-tts-ind' onto cpu...
MMS TTS model loaded successfully. Sample rate: 16000 Hz
✅ HiTeman Audio Therapy System (MMS TTS) ready!
🌟 HITEMAN AUDIO THERAPY SYSTEM DEMO (MMS TTS) 🌟

        🌟 SELAMAT DATANG DI HITEMAN AUDIO THERAPY 🌟

        Halo! Saya adalah asisten terapi audio digital yang akan membantu Anda
        menemukan ketenangan dan dukungan untuk kesehatan mental Anda.

        Sistem ini menggunakan teknologi AI voice (MMS TTS) untuk memberikan:
        • Audio

📁 Generated files for sedih:
   • therapy_mms_sedih_1.wav
   • therapy_mms_sedih_2.wav


🎵 DEMO 2: THERAPY SESSION - CEMAS
----------------------------------------
 Starting therapy session for: CEMAS
📏 Session length: short

        🎵 SESI TERAPI AUDIO - CEMAS 🎵

        Berdasarkan kondisi yang Anda alami, saya akan memberikan panduan terapi audio
        yang dirancang khusus untuk membantu mengatasi cemas.

        Silakan:
        • Duduk atau berbaring dengan nyaman
        • Gunakan headphone untuk pengalaman terbaik
        • Ikuti panduan yang diberikan dengan santai
        • Jangan memaksakan diri, lakukan sesuai kemampuan

        Mari kita mulai...
        
🎵 Generating 2 audio files for cemas therapy using MMS TTS...
   Creating audio 1/2...
Synthesizing with MMS TTS: 'Saya memahami bahwa kecemasan yang Anda rasakan sa...'
MMS Audio generated: 264960 samples at 16000 Hz
Applying post-processing (v2)
 Noise reduction (v2) applied.
   High-pass filter applied (cutoff: 90.0 

📁 Generated files for cemas:
   • therapy_mms_cemas_1.wav
   • therapy_mms_cemas_2.wav


🎵 DEMO 3: THERAPY SESSION - STRESS
----------------------------------------
 Starting therapy session for: STRESS
📏 Session length: short

        🎵 SESI TERAPI AUDIO - STRESS 🎵

        Berdasarkan kondisi yang Anda alami, saya akan memberikan panduan terapi audio
        yang dirancang khusus untuk membantu mengatasi stress.

        Silakan:
        • Duduk atau berbaring dengan nyaman
        • Gunakan headphone untuk pengalaman terbaik
        • Ikuti panduan yang diberikan dengan santai
        • Jangan memaksakan diri, lakukan sesuai kemampuan

        Mari kita mulai...
        
🎵 Generating 2 audio files for stress therapy using MMS TTS...
   Creating audio 1/2...
Synthesizing with MMS TTS: 'Saya dapat merasakan bahwa Anda sedang mengalami t...'
MMS Audio generated: 241152 samples at 16000 Hz
Applying post-processing (v2)
 Noise reduction (v2) applied.
   High-pass filter applied (cutoff: 

📁 Generated files for stress:
   • therapy_mms_stress_1.wav
   • therapy_mms_stress_2.wav


 COMPLETE!

Demo HiTeman Audio Therapy (MMS TTS) Selesai.
Terima kasih telah menggunakan HiTeman!
